In [16]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import psycopg2.extras as extras

# We are also going to do some basic viz
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pickle


In [21]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'fitness',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

connection = pg.connect(**connection_args)  # What is that "**" there??

### Download Data

Download Data from NHANES website (note that DXX only available from 2011 and on, prior to that had data issues and would require more data massaging)

In [ ]:
years = dict([('2011-2012','G'),('2013-2014','H'),('2015-2016','I'),\
                  ('2017-2018','J')])
data_type = ['DEMO','DR1TOT','PAQ','DBQ','WHQ','BMX','DXX']

Note that initial database was set up using psql and tables made for the 2017-2018 table, then used the code here to insert prior year survey data into the table.

Because DR1TOT files were not set up well to begin with, issues with appending with prior survey data, and therefore did not do so.

Also, because some fields in the 2017-2018 survey tables were newer, the code below will add new columns of 0s (initially Nulls, but had some issues halfway so switch to 0s.. but this should not impact analysis.) Therefore, prior to using any field, should do a test to see distribution by survey year to make sure that it is valid across all years.

additional table called "fat" was created that lists fitness category by gender and body fat range. (classes of "Essential Fat", "Athlete", "Fitness", "Average" (note originally called Acceptable), "Obese", as designated by the American Counsel on Exercise)

In [ ]:
import requests
         
for key in years:
    for data in data_type:
        url = "https://wwwn.cdc.gov/Nchs/Nhanes/"+key+"/"+data+"_"+years[key]+".XPT"
        r= requests.get(url)
        filename = 'Data/Raw/'+data+'_'+years[key]+'.XPT'
        with open(filename,'wb') as out_file:
            out_file.write(r.content)

In [ ]:
import xport.v56

In [ ]:
def execute_values(conn, df, table,columns):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    #make new df into same columns as existing table
    missing_cols = list(set(columns) - set(df.columns))
    new_df = df
    for i in missing_cols:
        blank_series = pd.Series([None]*len(df),name=i)
        new_df = pd.concat([new_df,blank_series],axis=1)
    fin_df = new_df[columns]
    
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in fin_df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(fin_df[columns]))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [ ]:
data_type = ['DEMO','PAQ','DBQ','WHQ','BMX','DXX']
survey = ['G','H','I']

for data in data_type:
    with open('Data/Raw/'+data+'_J.xpt', 'rb') as f:
        library = xport.v56.load(f)
    columns = list(library[data+'_J'].columns)

             
    for letter in survey:
        with open('Data/Raw/'+data+'_'+letter+'.xpt', 'rb') as f:
            library = xport.v56.load(f)
        execute_values(connection,library[data+'_'+letter],data,columns)

## EDA

In [22]:
query = '''
SELECT DISTINCT dxx.seqn, 
	CASE demo.riagendr WHEN '1.0' THEN 'M' WHEN '2.0' THEN 'F' ELSE demo.riagendr END, 
	demo.ridageyr, dxx.dxdtopf, paq.paq610, paq.paq625, paq.paq640, paq.paq655, paq.paq670, bmx.bmxbmi, fat.category, bmi_cat.bmi_category
FROM dxx
LEFT JOIN demo
	ON dxx.seqn = demo.seqn
LEFT JOIN paq
	ON dxx.seqn = paq.seqn
LEFT JOIN bmx
    ON dxx.seqn = bmx.seqn
JOIN fat
	ON demo.riagendr = fat.gender AND dxdtopf >= fat.low AND dxdtopf < fat.high
JOIN bmi_cat
    ON bmx.bmxbmi >= bmi_cat.low AND bmx.bmxbmi < bmi_cat.high
WHERE demo.ridageyr >= 18
ORDER BY dxx.seqn
'''

df = pd.read_sql(query, connection)


In [23]:
len(df)

11791

In [24]:
df

,seqn,riagendr,ridageyr,dxdtopf,paq610,paq625,paq640,paq655,paq670,bmxbmi,category,bmi_category
0,62161.0,M,22.0,24.3,NaN,NaN,NaN,NaN,NaN,23.3,Average,Normal
1,62164.0,F,44.0,25.9,5.0,NaN,NaN,5.0,1.0,23.2,Average,Normal
2,62169.0,M,21.0,19.8,NaN,NaN,NaN,NaN,NaN,20.1,Average,Normal
3,62172.0,F,43.0,42.2,NaN,NaN,NaN,NaN,NaN,33.3,Obese,Obese
4,62179.0,M,55.0,27.6,NaN,NaN,5.0,NaN,NaN,27.6,Obese,Overweight
...,...,...,...,...,...,...,...,...,...,...,...,...
11786,102935.0,F,27.0,33.7,3.0,NaN,4.0,1.0,1.0,23.6,Obese,Normal
11787,102944.0,M,55.0,33.4,NaN,5.0,NaN,NaN,NaN,29.6,Obese,Overweight
11788,102948.0,F,31.0,27.7,NaN,NaN,NaN,5.0,NaN,20.2,Average,Normal
11789,102949.0,M,33.0,15.5,3.0,7.0,7.0,NaN,NaN,23.7,Fitness,Normal


In [26]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [39]:
table = df.pivot_table(index='bmi_category',columns='category',aggfunc='count')['seqn'].copy()

column_order = ['Athlete','Fitness','Average','Obese']
index_order = ['Underweight','Normal','Overweight','Obese']


table.reindex(index = index_order, columns=column_order)

category,Athlete,Fitness,Average,Obese
bmi_category,,,,
Underweight,14.0,101.0,115.0,27.0
Normal,81.0,447.0,1552.0,1604.0
Overweight,5.0,53.0,658.0,2908.0
Obese,NaN,1.0,124.0,4101.0


Note that the BMI category is different than the (body fat) category. The body fat category is determined by the American Counsel on Exercise.

In [13]:
df.category.unique()

array(['Average', 'Obese', 'Fitness', 'Athlete'], dtype=object)

In [7]:
colnames = {
    "seqn": "id",
    "riagendr": "gender",
    "ridageyr": "age",
    "dxdtopf": "body_fat",
    "paq610": "vig_work_days",
    "paq625": "mod_work_days",
    "paq640": "walk_bike_days",
    "paq655": "vig_rec_days",
    "paq670": "mod_work_days"
}

In [8]:
df = df.rename(columns = colnames)

In [ ]:
df

In [10]:
df.isnull().sum()

id                   0
gender               0
age                  0
body_fat             0
vig_work_days     9037
mod_work_days     7105
walk_bike_days    8308
vig_rec_days      7972
mod_work_days     6558
category             0
dtype: int64

In [11]:
df = df.fillna(value=0)

In [12]:
with open('data/df_full.pickle', 'wb') as f:
    pickle.dump(df, f)